In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from sklearn.metrics import confusion_matrix, accuracy_score
import mysql.connector

In [3]:
# Koneksi ke database MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="",
    database="data_set"
)
cursor = conn.cursor()

In [6]:
# Preprocessing: Case Folding
cursor.execute("SELECT text, sentimen FROM training")
data = cursor.fetchall()
dataset = pd.DataFrame(data, columns=['text', 'sentimen'])
dataset['text'] = dataset['text'].str.lower()
dataset.head(10)



,text,sentimen
0,text,sentiment
1,ini beneran adalah serum pertama aku. pertama ...,Positif
2,pertama beli yg 20ml karena kepo sama serum in...,Negatif
3,saya rutin pake yang somethinc 5% selama 3 bul...,Positif
4,jadi sering banget liat seliweran kalo serum i...,Positif
5,sukaa bgtt sama produk satu ini!! awalnya ga b...,Positif
6,awal cobain produk ini karena lagi jerawatan d...,Negatif
7,produk something satupun tidak ada yang gagal ...,Positif
8,salah satu skincare tetap di regimenku. konsen...,Positif
9,"serum andalan aku banget ini, udah rutin bange...",Positif


In [20]:
# proses cleansing remove regex (cleansing) seperti tanda baca dan angka angka
import re
import string
def cleansing(text):
    text = text.strip(" ")
    text = re.sub(r'[?|$|.|!_:")(-+,]', '', text)
    text = re.sub(r'\d+', '', text)
    text = re.sub(r"\b[a-zA-Z]\b", "", text)
    text = re.sub('\s+',' ', text)
    return text
dataset['text'] = dataset['text'].apply(cleansing)
dataset.head(10)

,text,sentimen
0,text,sentiment
1,serum kali pake serum pas masih kelas sma pake...,Positif
2,beli yg ml sama serum dipake saat kondisi kuli...,Negatif
3,rutin pake lama bulan-an muka cerah bersih bai...,Positif
4,sering banget liat seliwer kalo serum bagus be...,Positif
5,suka banget sama satu gak harap tapi nyata bik...,Positif
6,cobain lagi jerawat review bilang bagus pakai ...,Negatif
7,something satu tidak ada gagal semua sangat ba...,Positif
8,salah satu tetap regimenku yg anjur dermatolog...,Positif
9,serum andal banget udah rutin banget pake coco...,Positif


In [19]:
# Replace specific words
replacement_dict = {
    r'\b(ga)\b': 'gak',
    r'\b(ngga|nggak|engga|enggak)\b': 'gak',
    r'\b(gaada)\b': 'gak ada',
    r'\b(pdhl)\b': 'padahal',
    r'\b(lgsg)\b': 'langsung',
    r'\b(bgtt{1,3})\b': 'banget',
    r'\b(banget{1,3})\b': 'banget',
    r'\b(stlh)\b': 'setelah',
    r'\b(sukaa{1,3})\b': 'suka',
    r'\b(seneng{2,3})\b': 'seneng'
}

def replace_words(text):
    for pattern, replacement in replacement_dict.items():
        text = re.sub(pattern, replacement, text)
    return text

dataset['text'] = dataset['text'].apply(replace_words)

In [21]:
#NLTK word tokenize
def word_tokenize_wrapper(text):
 return word_tokenize(text)
dataset['text'] = dataset['text'].apply(word_tokenize_wrapper)
dataset.head()

,text,sentimen
0,[text],sentiment
1,"[serum, kali, pake, serum, pas, masih, kelas, ...",Positif
2,"[beli, yg, ml, sama, serum, dipake, saat, kond...",Negatif
3,"[rutin, pake, lama, bulan-an, muka, cerah, ber...",Positif
4,"[sering, banget, liat, seliwer, kalo, serum, b...",Positif


In [24]:
# Preprocessing: Stopword Removal

custom_stopwords = ['adalah', 'adanya', 'adapun', 'agar', 'akan', 'akankah', 'akhir', 'akhiri', 'akhirnya', 'aku',
                    'akulah', 'amat', 'amatlah', 'anda', 'andalah', 'antar', 'antara', 'antaranya', 'apa', 'apaan',
                    'apabila', 'apakah', 'apatah', 'artinya', 'asal', 'asalkan', 'atas', 'atau', 'ataukah', 'ataupun',
                    'awal', 'awalnya', 'bagai', 'bagaikan', 'bagaimana', 'bagaimanakah', 'bagaimanapun', 'bagi', 'bagian',
                    'bahkan', 'bahwa', 'bahwasanya', 'baik', 'bakal', 'bakalan', 'balik', 'banyak', 'bapak', 'baru', 'bawah',
                    'beberapa', 'begini', 'beginian', 'beginikah', 'beginilah', 'begitu', 'begitukah', 'begitulah', 'begitupun',
                    'bekerja', 'belakang', 'belakangan', 'belumlah', 'benar', 'benarkah', 'benarlah', 'berada', 'berakhir',
                    'berakhirlah', 'berakhirnya', 'berapa', 'berapakah', 'berapalah', 'berapapun', 'berarti', 'berawal', 'berbagai',
                    'berdatangan', 'beri', 'berikan', 'berikut', 'berikutnya', 'berjumlah', 'berkali-kali', 'berkata', 'berkehendak',
                    'berkeinginan', 'berkenaan', 'berlainan', 'berlalu', 'berlangsung', 'berlebihan', 'bermacam', 'bermacam-macam',
                    'bermaksud', 'bermula', 'bersama', 'bersama-sama', 'bersiap', 'bersiap-siap', 'bertanya', 'bertanya-tanya',
                    'berturut', 'berturut-turut', 'bertutur', 'berujar', 'berupa', 'besar', 'betulkah', 'bila', 'bilakah', 'bisa',
                    'bisakah', 'bolehkah', 'buat', 'bukan', 'bukankah', 'bukanlah', 'bukannya', 'bulan', 'bung', 'cukupkah', 'dahulu',
                    'dalam', 'dan', 'dapat', 'dari', 'daripada', 'datang', 'dekat', 'demi', 'demikian', 'demikianlah', 'dengan',
                    'depan', 'di', 'dia', 'diakhiri', 'diakhirinya', 'dialah', 'diantara', 'diantaranya', 'diberi', 'diberikan',
                    'diberikannya', 'dibuat', 'dibuatnya', 'didapat', 'didatangkan', 'diibaratkan', 'diibaratkannya', 'diingat',
                    'diingatkan', 'diinginkan', 'dijawab', 'dijelaskan', 'dijelaskannya', 'dikarenakan', 'dikatakan', 'dikatakannya',
                    'dikerjakan', 'diketahui','diketahuinya','dilakukan','dilalui','dilihat','dimaksud','dimaksudkan','dimaksudkannya',
                    'dimaksudnya','diminta','dimintai','dimisalkan','dimulainya','dimungkinkan','dini','dipastikan','diperbuat',
                    'diperbuatnya', 'dipergunakan','diperkirakan','diperlihatkan','diperlukan','diperlukannya','dipersoalkan','dipunyai',
                    'diri','dirinya','disampaikan','disebut','disebutkan','disebutkannya','disini','disinilah','ditambahkan',
                    'ditandaskan','ditanya','ditanyai','ditanyakan','ditegaskan','ditujukan','ditunjuk','ditunjuki','ditunjukkan',
                    'ditunjukkannya','ditunjuknya','dituturkan', 'dituturkannya', 'diucapkan', 'diucapkannya', 'diungkapkan',
                    'dong', 'dulu', 'empat', 'gunakan', 'hal', 'hanya', 'hanyalah', 'hari', 'haruslah', 'hendak', 'hendaklah',
                    'hendaknya', 'hingga', 'ia', 'ialah', 'ibarat', 'ibaratkan', 'ibaratnya', 'ibu', 'ikut', 'ingat', 'ingat-ingat',
                    'ingin', 'inginkah', 'inginkan', 'ini', 'inikah', 'inilah', 'itu', 'itukah', 'itulah', 'jadi', 'jadilah',
                    'jadinya', 'jangan', 'jangankan', 'janganlah', 'jauh', 'jawab', 'jawaban', 'jawabnya', 'jelaskan', 'jelasnya',
                    'jika', 'jikalau', 'juga', 'jumlah', 'jumlahnya', 'kala', 'kalaulah', 'kalaupun', 'kalian', 'kami', 'kamilah',
                    'kamu', 'kamulah', 'kan', 'kapankah', 'kapanpun', 'karenanya', 'kasus', 'kata', 'katakan', 'katakanlah', 'katanya',
                    'ke', 'keadaan', 'kebetulan', 'kecil', 'kedua', 'keduanya', 'keinginan', 'kelima', 'keluar', 'kembali', 'kemudian',
                    'kepada', 'kepadanya', 'kesampaian', 'keseluruhan', 'keseluruhannya', 'keterlaluan', 'ketika', 'khususnya',
                    'kini', 'kinilah', 'kiranya', 'kitalah', 'kok', 'lah', 'lalu', 'lanjut', 'lanjutnya', 'lebih', 'lewat', 'lima',
                    'luar', 'macam', 'maka', 'makanya', 'mampukah', 'mana', 'manakala', 'manalagi', 'masa', 'masihkah', 'masing',
                    'masing-masing', 'maupun', 'melainkan', 'melakukan', 'melalui', 'melihat', 'melihatnya', 'memang', 'memastikan',
                    'memberi', 'memberikan', 'membuat', 'memerlukan', 'memihak', 'meminta', 'memintakan', 'memisalkan', 'memperbuat',
                    'mempergunakan', 'memperkirakan', 'memperlihatkan', 'mempersiapkan', 'mempersoalkan', 'mempertanyakan', 'mempunyai',
                    'memulai', 'memungkinkan', 'menaiki', 'menandaskan', 'menantikan', 'menanya', 'menanyai', 'menanyakan', 'mendapat',
                    'mendatang', 'mendatangi', 'mendatangkan', 'menegaskan', 'mengatakan', 'mengatakannya', 'mengenai', 'mengerjakan',
                    'mengetahui', 'menghendaki', 'mengibaratkan', 'mengibaratkannya', 'mengingat', 'mengingatkan', 'menginginkan',
                    'mengira', 'mengucapkan', 'mengucapkannya', 'mengungkapkan', 'menjawab', 'menjelaskan', 'menuju', 'menunjuk',
                    'menunjuki', 'menunjukkan', 'menunjuknya', 'menurut', 'menuturkan', 'menyampaikan', 'menyangkut', 'menyatakan',
                    'menyebutkan', 'menyeluruh', 'menyiapkan', 'mereka', 'merekalah', 'merupakan', 'meyakini', 'meyakinkan', 'minta',
                    'mirip', 'misal', 'misalkan', 'misalnya', 'mula', 'mulai', 'mulailah', 'mungkin', 'mungkinkah', 'nah', 'naik',
                    'nyaris', 'nyatanya', 'oleh', 'olehnya', 'pada', 'padanya', 'pak', 'paling', 'panjang', 'pantas', 'para', 'pasti',
                    'pastilah', 'penting', 'pentingnya', 'per', 'perlukah', 'perlunya', 'persoalan', 'pertama', 'pertanyaan',
                    'pertanyakan', 'pihak', 'pihaknya', 'pukul', 'pula', 'pun', 'punya', 'saatnya', 'saling', 'sama-sama', 'sambil',
                    'sampai', 'sampai-sampai', 'sampaikan', 'sana', 'saya', 'sayalah', 'se', 'sebab', 'sebabnya', 'sebagai',
                    'sebagaimana', 'sebagainya', 'sebagian', 'sebaik', 'sebaik-baiknya', 'sebaliknya', 'sebanyak', 'sebegini',
                    'sebegitu', 'sebelum', 'sebenarnya', 'seberapa', 'sebesar', 'sebisanya', 'sebuah', 'sebut', 'sebutlah',
                    'sebutnya', 'secara', 'sedemikian', 'seenaknya', 'segala', 'segalanya', 'seingat', 'sejauh', 'sejenak',
                    'sejumlah', 'sekadar', 'sekadarnya', 'sekali', 'sekalian', 'sekecil', 'seketika', 'sekiranya', 'sekitarnya',
                    'sekurang-kurangnya', 'sekurangnya', 'sela', 'selain', 'selaku', 'selanjutnya', 'seluruhnya', 'semacam', 'semampu',
                    'semampunya', 'semasa', 'semasih', 'semata', 'semata-mata', 'semaunya', 'sempat', 'sendirian', 'sendirinya', 'seolah',
                    'seolah-olah', 'seorang', 'sepanjang', 'sepantasnya', 'sepantasnyalah', 'seperlunya', 'seperti', 'sepertinya',
                    'sepihak', 'seringnya', 'serta', 'serupa', 'sesaat', 'sesama', 'sesampai', 'sesegera', 'seseorang', 'sesuatu',
                    'sesuatunya', 'sesudah', 'sesudahnya', 'setelah', 'setempat', 'setengah', 'seterusnya', 'setiap', 'setiba',
                    'setibanya', 'setidak-tidaknya', 'setidaknya', 'sela', 'selain', 'selaku', 'selanjutnya', 'seluruhnya', 'semacam',
                    'semampu', 'semampunya', 'semasa', 'semasih', 'semata', 'semata-mata', 'semaunya', 'sempat', 'sendirian',
                    'sendirinya', 'seolah', 'seolah-olah', 'seorang', 'sepanjang', 'sepantasnya', 'sepantasnyalah', 'seperlunya',
                    'seperti', 'sepertinya', 'sepihak', 'seringnya', 'serta', 'serupa', 'sesaat', 'sesama', 'sesampai', 'sesegera',
                    'seseorang', 'sesuatu', 'sesuatunya', 'sesudah', 'sesudahnya', 'setelah', 'setempat', 'setengah', 'seterusnya',
                    'setiap', 'setiba', 'setibanya', 'setidak-tidaknya', 'setidaknya', 'sila', 'silakan', 'sini', 'sinilah', 'soal',
                    'suatu', 'sudah', 'sudahkah', 'sudahlah', 'supaya', 'tadi', 'tadinya', 'tahu', 'tahun', 'tak', 'tambahnya',
                    'tampaknya', 'tandas', 'tandasnya', 'tanya', 'tanyakan', 'tanyanya', 'tegas', 'tegasnya', 'telah', 'tempat',
                    'tengah', 'tentang', 'tentu', 'tentulah', 'tentunya', 'tepat', 'terakhir', 'terasa', 'terbanyak', 'terdahulu',
                    'terdapat', 'terdiri', 'terhadap', 'terhadapnya', 'teringat', 'teringat-ingat', 'terjadilah', 'terjadinya',
                    'terkira', 'tersampaikan', 'tersebut', 'tersebutlah', 'tertentu', 'tertuju', 'tidakkah', 'tidaklah', 'tiga',
                    'tinggi', 'toh', 'tunjuk', 'turut', 'tutur', 'tuturnya', 'ucap', 'ucapnya', 'ujar', 'ujarnya', 'ungkap', 'ungkapnya',
                    'usai', 'waduh', 'wah', 'wahai', 'waktunya', 'walau', 'wong', 'yaitu', 'yakni', 'bener', 'beneran', 'pertama', 'SMA',
                    'aku', 'ini', 'buat', 'yang', 'dan', 'berbagai', 'setelah', 'guys', '10% niacinamide', '5% niacinamide', 'aja',
                    'pemakaian', 'kepo', 'penasaran', 'alhamdulillah', '1', '2', '3', 'kedua', 'ketiga', 'deh', 'besok', 'gue',
                    'produk', 'skincare', 'konsentrasi', 'member', 'adanya', 'karena', 'fd', 'female daily', 'somethinc', 'official',
                    'store', 'e-commerce', 'shopee', 'malam', 'niacin', 'mix', 'feeling', 'SMP', 'anak', 'to', 'jadiin', 'pol', 'vit',
                    'c', 'vitamin', 'pagi', 'paginya', 'online']

from nltk.corpus import stopwords
def stopword_removal(Review):
    filtering = stopwords.words('indonesian','english')
    filtering.extend(custom_stopwords)
    x = []
    data = []
    def myFunc(x):
        if x in filtering:
            return False
        else:
            return True
    fit = filter(myFunc, Review)
    for x in fit:
        data.append(x)
    return data
dataset['text'] = dataset['text'].apply(stopword_removal)
dataset.head(10)

,text,sentimen
0,[text],sentiment
1,"[serum, kali, pake, serum, pas, kelas, sma, pa...",Positif
2,"[beli, yg, ml, serum, dipake, kondisi, kulit, ...",Negatif
3,"[rutin, pake, bulan-an, muka, cerah, bersih, s...",Positif
4,"[banget, liat, seliwer, kalo, serum, bagus, be...",Positif
5,"[suka, banget, gak, harap, nyata, bikin, cerah...",Positif
6,"[cobain, jerawat, review, bilang, bagus, pakai...",Negatif
7,"[something, gagal, bagus, masuk, suka, cari, c...",Positif
8,"[salah, regimenku, yg, anjur, dermatologist, t...",Positif
9,"[serum, andal, banget, udah, rutin, banget, pa...",Positif


In [26]:
def stemming(text):
    factory = StemmerFactory()
    stemmer = factory.create_stemmer()
    do = []
    for w in text:
        dt = stemmer.stem(w)
        do.append(dt)
    d_clean = " ".join(do)
    print(d_clean)
    return d_clean

dataset['text'] = dataset['text'].apply(stemming)
dataset.to_csv('datareviewbersih.csv', index=False)
data_clean = pd.read_csv('datareviewbersih.csv', encoding='latin1')
data_clean.head(10)


text
serum kali pake serum pas kelas sma pake jerawat parah banget fungal acne bruntusan komedo full penuh wajah
beli yg ml serum dipake kondisi kulit lg fine fine few tiny bumps unfortunately langsung ngasih effect panas kulit dipake panas yg bikin itch
rutin pake bulan-an muka cerah bersih skin barier gampang jerawat muka mulus gitu thanks
banget liat seliwer kalo serum bagus beli dehhh work banget minggu pake udah bikin cerah muka gak bikin lembab sih gak bikin kering yaa aman aman kal
suka banget gak harap nyata bikin cerah skin barrier gak gampang jerawat tesktur kulit halus repurchase kar
cobain jerawat review bilang bagus pakai asa panas kulit langsung nongol jerawat batu jerawat muncul emang
something gagal bagus masuk suka cari cantik something pilih ragu membuktik
salah regimenku yg anjur dermatologist tekstur cair kental warna beninh mudah serap lengket kulit niacinami
serum andal banget udah rutin banget pake cocok serum enak texturenya gak cair gak kental mudah aplikasikan

,text,sentimen
0,text,sentiment
1,serum kali pake serum pas kelas sma pake jeraw...,Positif
2,beli yg ml serum dipake kondisi kulit lg fine ...,Negatif
3,rutin pake bulan-an muka cerah bersih skin bar...,Positif
4,banget liat seliwer kalo serum bagus beli dehh...,Positif
5,suka banget gak harap nyata bikin cerah skin b...,Positif
6,cobain jerawat review bilang bagus pakai asa p...,Negatif
7,something gagal bagus masuk suka cari cantik s...,Positif
8,salah regimenku yg anjur dermatologist tekstur...,Positif
9,serum andal banget udah rutin banget pake coco...,Positif


In [28]:
import string
from sklearn.pipeline import Pipeline
import pandas as pd
import numpy as np
import re
data_clean = pd.read_csv('datareviewbersih.csv', encoding='latin1')
data_clean.head()

,text,sentimen
0,text,sentiment
1,serum kali pake serum pas kelas sma pake jeraw...,Positif
2,beli yg ml serum dipake kondisi kulit lg fine ...,Negatif
3,rutin pake bulan-an muka cerah bersih skin bar...,Positif
4,banget liat seliwer kalo serum bagus beli dehh...,Positif


In [51]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# Pembagian Dataset menjadi Data Training dan Testing
X = dataset['text']
y = dataset['sentimen']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Create the CountVectorizer object
vectorizer = CountVectorizer()

# Fit and transform the training data
X_train_vectorized = vectorizer.fit_transform(X_train)

# Transform the testing data
X_test_vectorized = vectorizer.transform(X_test)

# Create the Naive Bayes classifier
nb_classifier = MultinomialNB()

# Fit the classifier to the training data
nb_classifier.fit(X_train_vectorized, y_train)

# Evaluate the model
y_pred = nb_classifier.predict(X_test_vectorized)


In [52]:
# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:")
print(cm)

# Accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Confusion Matrix:
[[44 14  4]
 [12 27 14]
 [ 6 22 40]]
Accuracy: 0.6065573770491803


In [53]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

     Negatif       0.71      0.71      0.71        62
      Netral       0.43      0.51      0.47        53
     Positif       0.69      0.59      0.63        68

    accuracy                           0.61       183
   macro avg       0.61      0.60      0.60       183
weighted avg       0.62      0.61      0.61       183

